# Imports

In [1]:
%pip install --upgrade tiktoken
%pip install --upgrade sentencepiece
%pip install --upgrade tensorflow
%pip install --upgrade tf_keras
%pip install --upgrade transformers
%pip install ml_dtypes==0.5.1 # it appears some code does not work if I don't downgrade this library
%pip install --upgrade minicons

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Note: you may need to restart the kernel to use updated packages.


In [2]:
#import tensorflow
#import pandas as pd
import numpy as np
#from transformers import pipeline
#from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
#import torch
import huggingface_hub as hgh
from minicons import scorer
import operator

c:\Users\borah\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Login (for HuggingFace)

In [3]:
login_token_file = open("access_items/hg_token.txt", "r")
token = login_token_file.readline()
login_token_file.close()
hgh.login(token)

# Loading data for testing

In [4]:
def load_sentences(filename):
    """Loads sentences from a file formatted as: 
    context,translation
        completion1,translation,is_grammatical
        completion2,translation,is_grammatical
        ...
    """
    out = dict()
    with open(filename, "r", encoding="utf-8") as file:
        cur_header = None
        blank_index = int(1)
        for raw_line in file:
            if (raw_line.lstrip().startswith("#")):
                continue
            if (len(raw_line.strip()) == 0):
                continue
            line = raw_line.rstrip()
            if (line == "\\nil"):
                cur_header = blank_index
                out[blank_index] = []
                blank_index += 1
            elif line.startswith("\t") or line.startswith(" "*4):
                if not cur_header:
                    raise Exception(f"File '{filename}' in incorrect format at line {line}.")
                out[cur_header].append(tuple(line.lstrip().split(",")))
            else:
                translation_pair = tuple(line.split(","))
                out[translation_pair] = []
                cur_header = translation_pair
    return out

In [5]:
## the sentence sets for English (just to test as control)
english_contextual_sentences = {
    ("A boy and a girl walked into the classroom.",)*2 : [("A boy sat down.",)*2, ("The boy sat down.",)*2],
    ("I am expecting a student to come to class today.",)*2 : [("Has a student arrived?",)*2, ("Has the student arrived?",)*2],
    ("There once was a massive tree in the forest.",)*2 : [("A tree fell down.",)*2, ("The tree fell down.",)*2],
    ("Two men were talking.",)*2 : [("The man said a joke.",)*2, ("One man said a joke.",)*2]
}

## load in assamese sentence sets
assamese_contextual_sentences = load_sentences("test_sentences/assamese.txt")
print(assamese_contextual_sentences)

{('এজন ল’ৰা আৰু এজনী ছোৱালী এটা কোঠাৰ ভিতৰত সোমাই গ’ল।', 'A boy and a girl entered a room.'): [('ল’ৰাজন বহিল।', 'The boy sat down.', '1'), ('এজন ল’ৰা বহিল।', 'A boy sat down.', '0')], ('টেবুলখনত দুটা মেকুৰী আছে।', 'There are two cats on the table.'): [('এটা মেকুৰী তললৈ জপিয়াই পৰে।', 'One cat jumps down.', '1'), ('মেকুৰীটোৱে তললৈ জপিয়াই পৰে', 'The cat jumps down.', '0')], ('দুজন মানুহে কথা পাতি আছিল।', 'Two men were talking.'): [('এজন মানুহে গান গাবলৈ ধৰিলে।', 'One man started to sing.', '1'), ('মানুহজনে গান গাবলৈ ধৰিলে।', 'The man started to sing.', '0')], 1: [('টেবুলখনত দুটা মেকুৰী আছে।', 'There are two cats on the (preferred classifier) table.', '1'), ('টেবুলজনত দুটা মেকুৰী আছে।', 'There are two cats on the (incorrect classifier) table.', '0')]}


(Citations)

* (<em>AxomiyaBERTa</em>) [AxomiyaBERTa: A Phonologically-aware Transformer Model for Assamese](https://aclanthology.org/2023.findings-acl.739/) (Nath et al., Findings 2023) 

* (<em>IndicBERT</em>) [IndicNLPSuite: Monolingual Corpora, Evaluation Benchmarks and Pre-trained Multilingual Language Models for Indian Languages](https://aclanthology.org/2020.findings-emnlp.445/) (Kakwani et al., Findings 2020)

* (<em>XLM-R</em>) [Unsupervised Cross-lingual Representation Learning at Scale](https://aclanthology.org/2020.acl-main.747/) (Conneau et al., ACL 2020)


In [6]:
# all models we will use
all_models = {  "AxomiyaBERTa" : "Abhijnan/AxomiyaBERTa", #
                "IndicBERT" : "ai4bharat/indic-bert",
                "XLM-R" : "FacebookAI/xlm-roberta-base"}

# so we can easily swap out language documents used
language_documents = assamese_contextual_sentences

# average number of "correct choices" per model
surprisal_avg_rats = {}

# iterate thorugh models
for (model_name, model_path) in all_models.items():
    print(f'Loading model "{model_name}".')

    # use minicons scorer for surprisal
    scorer_model = scorer.MaskedLMScorer(model_path, "cpu")

    # number of contexts with the sentence that has min surprisal being the most grammatically correct option 
    surprisal_corrects = 0

    # get each set of sentences with their context
    for (context, sentences) in language_documents.items():

        context_blank = None 
        if (type(context) != tuple): # check if our context is supposed to be blank
            context = ("(no context)", f"({context})")
            context_blank = context
        
        #print(f'  Context sentence: \"{context[0]}\" (translation:\"{context[1]}\")')

        pairs = [(context, sentence) for sentence in sentences]

        # conjoin sentence of context with completion 
        # (the models don't seem to support sequencing surprisal 
        # so we will just concatenate the sentences together directly, unless context is '\nil')
        documents = [(("" if context_blank else (ctxt[0] + " ")) + sentence[0]) for (ctxt, sentence) in pairs]

        # get surprisal values
        output = scorer_model.sequence_score(documents, reduction = lambda x: -x.sum(0).item())

        # put the sentences and their surprisals in a dictionary
        surprisal_dict = (dict(zip([(sent[0],sent[1]) for (c, sent) in pairs], output)))
        #print("\t> Surprisal values for each sentence (alongside translations)=\n\t\t" + '\n\t\t'.join([f"'{sent}' (trans:'{trans}'): {surp}" for ((sent, trans), surp) in surprisal_dict.items()]))
        
        # get the sentence that got the minimum surprisal in this context
        min_surprisal_sentence = min(surprisal_dict.items(), key=operator.itemgetter(1))[0]
        #print(f"\t> Sentence with minimum surprisal for this context={min_surprisal_sentence}")
        
        if (len(pairs[0][1]) > 2):
            # get which completion we expected to get minimum surprisal (i.e. the "most correct" one)
            expected_min = max([s for (c, s) in pairs], key=operator.itemgetter(2))[0:2]
            #print(f"\t> Sentence expected to have min surprisal={expected_min}")
            
            # add it to the count
            if (surprisal_dict[min_surprisal_sentence] == surprisal_dict[expected_min]):
                surprisal_corrects += 1
    
    # store proportion of "correct values"
    surprisal_avg_rats[model_name] = surprisal_corrects / len(language_documents)

Loading model "AxomiyaBERTa".
Loading model "IndicBERT".
Loading model "XLM-R".


In [8]:
# print out the proportion of correct minimum surprisals
print(surprisal_avg_rats)

{'AxomiyaBERTa': 0.25, 'IndicBERT': 0.75, 'XLM-R': 0.75}
